In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD

### 1. 讀入 MNSIT 數據集

In [3]:
from tensorflow.keras.datasets import mnist

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

### 2. 資料整理

### Channel

CNN 要注意一張圖有多少個 channel, 開始我們因為只有灰階, 所以只有一個 channel。因此我們要轉一下我們的資料格式:

    (28,28) --> (28, 28, 1)

In [5]:
x_train = x_train.reshape(60000, 28, 28, 1) / 255

In [6]:
x_test = x_test.reshape(10000, 28, 28, 1) / 255

In [7]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

### 3. step 1: 打造函數學習機 (CNN)

In [8]:
model = Sequential()

In [9]:
model.add(Conv2D(32, (3,3), padding='same', strides=(1,1),
                input_shape=(28,28,1),
                activation='relu'))

In [10]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [11]:
model.add(Conv2D(64, (3,3), padding='same', strides=(1,1),
                activation='relu'))

In [12]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [13]:
model.add(Conv2D(128, (3,3), padding='same',
                activation='relu'))

In [14]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [15]:
model.add(Flatten())

In [16]:
model.add(Dense(870, activation='relu'))

In [17]:
model.add(Dense(600, activation='relu'))

In [18]:
model.add(Dense(10, activation='softmax'))

#### 看一下我們的神經網路

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0

#### 組裝

In [20]:
model.compile(loss='mse', optimizer=SGD(lr=0.123),
             metrics=['accuracy'])

### 4. step 2: fit

In [21]:
model.fit(x_train, y_train, batch_size=354, epochs=18)

Epoch 1/18
170/170 [==============================] - 2s 12ms/step - loss: 0.0898 - accuracy: 0.1762
Epoch 2/18
170/170 [==============================] - 2s 11ms/step - loss: 0.0893 - accuracy: 0.3682
Epoch 3/18
170/170 [==============================] - 2s 11ms/step - loss: 0.0886 - accuracy: 0.4253
Epoch 4/18
170/170 [==============================] - 2s 11ms/step - loss: 0.0872 - accuracy: 0.4691
Epoch 5/18
170/170 [==============================] - 2s 11ms/step - loss: 0.0819 - accuracy: 0.5014
Epoch 6/18
170/170 [==============================] - 2s 11ms/step - loss: 0.0514 - accuracy: 0.6949
Epoch 7/18
170/170 [==============================] - 2s 11ms/step - loss: 0.0267 - accuracy: 0.8303
Epoch 8/18
170/170 [==============================] - 2s 11ms/step - loss: 0.0193 - accuracy: 0.8759
Epoch 9/18
170/170 [==============================] - 2s 11ms/step - loss: 0.0151 - accuracy: 0.9030
Epoch 10/18
170/170 [==============================] - 2s 11ms/step - loss: 0.0125 - accura

### Step 3. 預測

In [22]:
result = model.predict_classes(x_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


### 看看測試資料表現如何

In [23]:
loss, acc = model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0059 - accuracy: 0.9620


In [24]:
print(f'測試資料的正確率為 {acc*100:.2f}%')

測試資料的正確率為 96.20%


In [25]:
def my_predict(n):
    print('我可愛的 CNN 預測是', result[n])
    X = x_test[n].reshape(28,28)
    plt.imshow(X, cmap='summer')

In [26]:
from ipywidgets import interact_manual

In [27]:
interact_manual(my_predict, n=(0, 9999));

interactive(children=(IntSlider(value=4999, description='n', max=9999), Button(description='Run Interact', sty…

### 把我們的 model 存起來

In [28]:
model.save('myCNNmodel.h5')